## ✋ Step 2: Hand Landmark Collection with MediaPipe

In [1]:

import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def collect_data(label, num_samples=100):
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
    cap = cv2.VideoCapture(0)
    collected = 0
    data = []

    print(f"✋ Showing webcam feed... Please make the gesture for '{label}'")

    while collected < num_samples:
        ret, frame = cap.read()
        if not ret:
            continue

        image = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                row = []
                for lm in hand_landmarks.landmark:
                    row.extend([lm.x, lm.y, lm.z])
                row.append(label)
                data.append(row)
                collected += 1
                print(f"Collected {collected}/{num_samples}", end="\r")
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("Data Collection - Press Q to Quit", image)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    return data


## 💾 Step 3: Save Landmarks to CSV

In [ ]:

import pandas as pd


label = "A" # Change this to the desired label to all letters
data = collect_data(label, 100)
df = pd.DataFrame(data)
df.to_csv(f"{label}_data.csv", index=False)


## 📂 Step 4: Combine All Letter CSV Files

In [3]:

import glob

csv_files = glob.glob("*_data.csv")
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_csv("asl_full_data.csv", index=False)


NameError: name 'pd' is not defined

## 🧠 Step 5: Train a Classifier (RandomForest)

In [4]:

from sklearn.ensemble import RandomForestClassifier
import pickle

def train_model(csv_path, model_path):
    df = pd.read_csv(csv_path)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    clf = RandomForestClassifier()
    clf.fit(X, y)
    with open(model_path, "wb") as f:
        pickle.dump(clf, f)
    print("✅ Model trained and saved!")

train_model("asl_full_data.csv", "asl_model.pkl")


NameError: name 'pd' is not defined

## 🎯 Step 6: Real-Time Prediction and Build Words

In [5]:

def live_prediction(model_path):
    import time
    with open(model_path, "rb") as f:
        model = pickle.load(f)

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
    mp_drawing = mp.solutions.drawing_utils

    cap = cv2.VideoCapture(0)
    word = ""
    cooldown = 1.5
    last_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        image = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                row = []
                for lm in hand_landmarks.landmark:
                    row.extend([lm.x, lm.y, lm.z])
                if len(row) == 63:
                    prediction = model.predict([row])[0]
                    current_time = time.time()
                    if current_time - last_time > cooldown:
                        word += prediction
                        last_time = current_time

                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.putText(image, f"Word: {word}", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Live Prediction - Press X to Quit", image)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("x") or len(word) >= 15:
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"📝 Final Word: {word}")


In [6]:
live_prediction("asl_model.pkl")



c:\Users\omarm\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\omarm\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\omarm\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\omarm\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\omarm\anaconda3\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\omarm\anaconda3\lib\site-packages\sklearn\base.py:4

📝 Final Word: FBOORDOOOOOEOEC


In [10]:
import pandas as pd
train_model("asl_full_data.csv", "asl_model.pkl")


✅ Model trained and saved!


In [11]:
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load data
df = pd.read_csv("asl_full_data.csv")
X = df.iloc[:, :-1]  # Features
y_true = df.iloc[:, -1]  # True labels

# Load trained model
with open("asl_model.pkl", "rb") as f:
    model = pickle.load(f)


In [12]:
# Predict on the dataset
y_pred = model.predict(X)

# Print accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")


✅ Model Accuracy: 100.00%
